In [18]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
  tf.config.experimental.set_memory_growth(gpu, True)

In [19]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import random

In [20]:
tf.keras.backend.clear_session()

In [21]:
subject = 'Amapá - Consumo de Cimento (t)'
start_index = 0
split_index = 191 #Referente aos 230 anos de input 
window_size = 36
train_split = split_index + 1 -30

In [22]:
data = pd.read_csv('2003_mo_model_input_AP.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Unnamed: 0,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),...,Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Year,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego,Amapá - Consumo de Cimento (t)
0,2003-1,0.711421,0.724032,11.520143,1.611498,1.036534,1.772069e+08,7.330309e+06,0.969649,1.911713,...,22.342619,83.376791,0.991074,2002.738651,8.035813e+06,356591.003430,10.883143,7.033593e+06,8.514392,10.392
1,2003-2,0.711553,0.690297,11.189862,1.450389,0.993449,1.773884e+08,7.335910e+06,0.950783,1.911107,...,22.345445,83.386754,0.991161,2002.769719,8.046432e+06,356647.711004,10.885206,7.037356e+06,8.508753,6.857
2,2003-3,0.711685,0.669681,10.820792,1.870184,0.973020,1.775699e+08,7.341511e+06,0.938332,1.910754,...,22.347093,83.392567,0.991211,2002.787842,8.057052e+06,356704.418579,10.887268,7.041120e+06,8.503114,7.011
3,2003-4,0.711817,0.660494,10.417840,1.355287,0.940489,1.777514e+08,7.347112e+06,0.926401,1.910716,...,22.347270,83.393189,0.991217,2002.789784,8.067671e+06,356761.126153,10.889331,7.044883e+06,8.497475,7.122
4,2003-5,0.711949,0.648337,9.959690,1.585538,0.917493,1.779329e+08,7.352713e+06,0.951683,1.916050,...,22.352711,83.458581,0.991472,2002.830344,8.078290e+06,356817.833728,10.891394,7.048646e+06,8.491835,5.267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,2022-8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.208
236,2022-9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.476
237,2022-10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.236
238,2022-11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.549


In [23]:
input_data = data.iloc[:split_index + 1,1:-1]
mean = np.mean(input_data, axis=0)
stddev =  np.std(input_data, axis=0)
input_data = ((input_data - mean) /stddev)
input_data

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Year,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.116290,-0.087050,...,0.232704,-1.452297,-1.147201,-0.368110,-1.732276,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.117269,-0.085088,...,0.233447,-1.446179,-1.144323,-0.367199,-1.725450,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.117840,-0.083944,...,0.233881,-1.442611,-1.142645,-0.366667,-1.721468,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.117902,-0.083821,...,0.233928,-1.442228,-1.142465,-0.366610,-1.721042,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.109278,-0.094674,...,0.223140,-1.430447,-1.123578,-0.363928,-1.712131,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.153927,0.757520,...,0.347008,-0.009983,-0.113276,-1.423514,1.616326,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.469742,0.567388,...,0.266950,0.068618,0.020019,-1.183686,1.617752,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.628813,0.668085,...,0.096485,0.310156,0.331876,-1.040777,1.650093,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.244795,0.790567,...,-0.207010,0.604742,0.792518,-0.709156,1.686595,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [24]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-12)
target_data

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
235       NaN
236       NaN
237       NaN
238       NaN
239       NaN
Name: Amapá - Consumo de Cimento (t), Length: 240, dtype: float64

In [25]:
# input para treinamento
train_input = input_data.iloc[start_index:train_split]
train_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Year,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
0,-2.943242,2.723741,4.398348,2.132459,3.890153,-2.042341,-2.389042,3.122582,-0.116290,-0.087050,...,0.232704,-1.452297,-1.147201,-0.368110,-1.732276,-1.716825,-0.487709,-0.246236,-1.984303,-0.890357
1,-2.870364,2.350880,4.222509,1.592202,3.551840,-2.014760,-2.352139,2.970356,-0.117269,-0.085088,...,0.233447,-1.446179,-1.144323,-0.367199,-1.725450,-1.697794,-0.479493,-0.208006,-1.951397,-0.891922
2,-2.797485,2.123016,4.026019,2.999926,3.391423,-1.987179,-2.315236,2.869895,-0.117840,-0.083944,...,0.233881,-1.442611,-1.142645,-0.366667,-1.721468,-1.678763,-0.471277,-0.169777,-1.918491,-0.893486
3,-2.724606,2.021477,3.811492,1.273293,3.135979,-1.959598,-2.278333,2.773628,-0.117902,-0.083821,...,0.233928,-1.442228,-1.142465,-0.366610,-1.721042,-1.659732,-0.463061,-0.131548,-1.885585,-0.895051
4,-2.651728,1.887113,3.567576,2.045407,2.955412,-1.932017,-2.241431,2.977624,-0.109278,-0.094674,...,0.223140,-1.430447,-1.123578,-0.363928,-1.712131,-1.640701,-0.454845,-0.093319,-1.852679,-0.896616
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,1.388995,-0.214006,-0.607704,0.043807,-1.233012,1.031384,0.819304,-0.883659,0.187573,0.988039,...,0.326030,1.583621,0.517601,-0.317463,1.133922,1.179199,-1.457028,-1.470096,1.047494,1.300799
158,1.332036,-0.434717,-0.620523,0.210856,-1.299304,1.042716,0.808136,-0.950771,0.478142,1.249068,...,0.302109,1.696590,0.612242,-0.730821,1.149126,1.184540,-1.486109,-1.464204,1.056372,1.292722
159,1.275077,-0.524091,-0.631530,0.107070,-1.248662,1.054049,0.796969,-1.028465,0.773595,1.361811,...,0.258553,1.668270,0.638407,-1.311677,1.160585,1.189880,-1.515190,-1.458311,1.065250,1.284645
160,1.218118,-0.614500,-0.640320,0.392942,-1.068274,1.065381,0.785801,-1.103668,0.690656,1.598360,...,0.365124,1.456065,0.377696,-1.584435,1.166946,1.195221,-1.544271,-1.452419,1.074128,1.276568


In [26]:
# Alvo para treinamento
train_target = target_data.iloc[start_index:train_split]
train_target

0      10.000
1       6.136
2       5.122
3       3.926
4       6.197
        ...  
157     8.322
158    11.373
159     9.651
160    10.855
161    13.970
Name: Amapá - Consumo de Cimento (t), Length: 162, dtype: float64

In [27]:
def create_batches(t_input, t_target, window_size, start_from):
    
    X_batches = []
    y_batches = []

    train_input_values = t_input.values 

    for i in range(len(t_input) - window_size):
        
        X_window = train_input_values[i:i+window_size, :]
        y_target = t_target[start_from+i+window_size]

        X_batches.append(X_window)
        y_batches.append(y_target)

    return np.array(X_batches), np.array(y_batches)

In [28]:
reshaped_train, reshaped_target = create_batches(train_input, 
                                                 train_target, 
                                                 window_size, 
                                                 start_index)
reshaped_train.shape

(126, 36, 21)

In [29]:
# input para treinamento
test_input = input_data.iloc[train_split-window_size:split_index + 1]
test_input

,Amapá - IDH,IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),INCC (%),Precipitation (mm/day),Atmospheric Pressure(mB),...,Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Year,Amapá - PIB - Estadual,Amapá - PIB - Construção Civil,Amapá - PIB - Per Capita,Amapá - PIB - Preços de Mercado,Amapá - Desemprego
126,0.639896,0.888984,-0.460555,-1.048761,-0.368821,0.651397,0.944085,0.045243,-0.726986,1.008561,...,1.932602,2.118009,-0.892570,1.269695,0.515782,0.813802,0.876851,-0.305631,0.923476,0.978734
127,0.580720,0.954254,-0.440372,-0.649312,-0.328087,0.664707,0.947319,0.061828,-0.973912,0.658311,...,2.516500,2.600626,-1.217426,1.959626,0.517825,0.829759,0.846024,-0.368234,0.925918,0.993895
128,0.521544,1.045217,-0.419247,-0.535369,-0.176031,0.678017,0.950553,0.046225,-1.033860,0.466149,...,2.589442,2.823380,-1.149632,2.392173,0.519250,0.845716,0.815197,-0.430837,0.928361,1.009056
129,0.462368,1.176395,-0.397019,-1.288807,-0.113037,0.691327,0.953786,0.032522,-1.120497,0.375368,...,1.965735,2.404365,-0.737101,1.851137,0.551592,0.861673,0.784370,-0.493440,0.930803,1.024217
130,0.403191,1.303259,-0.376532,-1.507550,-0.022703,0.704637,0.957020,0.042757,-1.255435,0.248427,...,1.213092,1.992627,-0.174400,1.562466,0.588093,0.877629,0.753543,-0.556042,0.933246,1.039378
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,-0.638974,-2.010387,-0.572934,-1.311366,0.589021,1.365911,0.389193,-1.749976,0.153927,0.757520,...,0.347008,-0.009983,-0.113276,-1.423514,1.616326,1.145925,-1.614044,-1.566534,1.010496,1.085785
188,-0.739823,-1.870713,-0.588777,-1.417143,1.043728,1.376610,0.370392,-1.593005,0.469742,0.567388,...,0.266950,0.068618,0.020019,-1.183686,1.617752,1.135411,-1.605628,-1.579697,0.995294,1.078764
189,-0.840672,-1.806230,-0.612606,-1.597784,1.387010,1.387308,0.351592,-1.351489,0.628813,0.668085,...,0.096485,0.310156,0.331876,-1.040777,1.650093,1.124897,-1.597213,-1.592859,0.980093,1.071742
190,-0.941521,-1.727496,-0.640956,-1.400787,1.815728,1.398006,0.332791,-1.198492,0.244795,0.790567,...,-0.207010,0.604742,0.792518,-0.709156,1.686595,1.114383,-1.588797,-1.606022,0.964891,1.064721


In [30]:
reshaped_test, reshaped_test_target = create_batches(test_input, 
                                                     target_data, 
                                                     window_size, 
                                                     train_split - window_size)
reshaped_test_target

array([12.399, 11.268,  8.741, 12.358, 11.876, 11.713, 11.449,  7.805,
        7.726,  8.516,  8.503, 10.175, 11.775,  9.724, 10.069, 13.93 ,
       11.918, 11.757,  9.691,  7.378,  6.97 ,  7.612,  8.442,  8.835,
       11.251, 11.342, 13.11 , 13.195, 10.053, 12.274])

In [31]:
def validation_splitter(arr, div_factor, add_factor=0):
    split_factor = len(arr) // div_factor
    positions_to_drop_index = []
    positions_to_drop = []
    for i in range(split_factor):
        pos = len(arr) - (i * div_factor + 1)
        positions_to_drop_index.append(pos)
        positions_to_drop.append(pos + add_factor)
    
    arr_droped = arr[positions_to_drop]
    arr_result = np.delete(arr, positions_to_drop_index, axis=0)
    
    return arr_result, arr_droped

In [32]:
# Rede Neural Recorrente com optmizador Estocástico
def lstm_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    train, train_val = validation_splitter(train_input, 7)
    target,target_val = validation_splitter(train_target, 7)
#     display(train.shape)
#     display(train_val.shape)
#     display(target.shape)
#     display(target_val.shape)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.LSTM(36, activation='tanh', 
                             return_sequences=True, 
                             input_shape=(reshaped_train.shape[1],
                                          reshaped_train.shape[2])),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.LSTM(180, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train, 
                        target, 
                        epochs=10000,
                        validation_data=(train_val,
                                         target_val),
                        callbacks=[early_stopping], 
                        verbose=want_verbose)
    return model, history

In [33]:
def get_a_good_seed(train_input, train_target, test_input, test_target):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(5)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = lstm_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [34]:
winner_seed = get_a_good_seed(reshaped_train, reshaped_target, reshaped_test, reshaped_test_target)
print(f"\n\nfinal_seed: {winner_seed}")

[3719565274, 3137244144, 3721634792, 2987373335, 2613665703]


Step: 0 ___________________________________________
val_loss: 6.051088333129883
winner_seed: 3719565274


Step: 1 ___________________________________________
val_loss: 4.955893516540527
winner_seed: 3137244144


Step: 2 ___________________________________________
val_loss: 5.603538513183594


Step: 3 ___________________________________________
val_loss: 5.431237697601318


Step: 4 ___________________________________________
val_loss: 5.652494430541992


final_seed: 3137244144


In [35]:
trained_model, history = lstm_model(reshaped_train, 
                                    reshaped_target, 
                                    want_verbose=1, 
                                    seed=winner_seed)

Epoch 1/10000
4/4 [==============================] - 2s 130ms/step - loss: 230.2701 - val_loss: 136.0424
Epoch 2/10000
4/4 [==============================] - 0s 10ms/step - loss: 66.2376 - val_loss: 44.7894
Epoch 3/10000
4/4 [==============================] - 0s 11ms/step - loss: 26.0041 - val_loss: 33.7152
Epoch 4/10000
4/4 [==============================] - 0s 11ms/step - loss: 21.2068 - val_loss: 25.3307
Epoch 5/10000
4/4 [==============================] - 0s 10ms/step - loss: 15.5708 - val_loss: 21.0451
Epoch 6/10000
4/4 [==============================] - 0s 10ms/step - loss: 13.4297 - val_loss: 27.2801
Epoch 7/10000
4/4 [==============================] - 0s 10ms/step - loss: 21.5344 - val_loss: 19.3394
Epoch 8/10000
4/4 [==============================] - 0s 10ms/step - loss: 12.0688 - val_loss: 15.3632
Epoch 9/10000
4/4 [==============================] - 0s 11ms/step - loss: 9.8862 - val_loss: 33.1359
Epoch 10/10000
4/4 [==============================] - 0s 10ms/step - loss: 11.78

4/4 [==============================] - 0s 11ms/step - loss: 5.1388 - val_loss: 8.9417
Epoch 82/10000
4/4 [==============================] - 0s 12ms/step - loss: 5.1985 - val_loss: 14.0092
Epoch 83/10000
4/4 [==============================] - 0s 11ms/step - loss: 6.1544 - val_loss: 10.1563
Epoch 84/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.6261 - val_loss: 10.9598
Epoch 85/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3108 - val_loss: 11.0812
Epoch 86/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.2669 - val_loss: 11.2663
Epoch 87/10000
4/4 [==============================] - 0s 9ms/step - loss: 5.7877 - val_loss: 11.5577
Epoch 88/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.4875 - val_loss: 10.4711
Epoch 89/10000
4/4 [==============================] - 0s 10ms/step - loss: 6.2242 - val_loss: 9.4780
Epoch 90/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1383 - val_loss: 13.414

4/4 [==============================] - 0s 11ms/step - loss: 5.9412 - val_loss: 9.1071
Epoch 162/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.8048 - val_loss: 9.7873
Epoch 163/10000
4/4 [==============================] - 0s 11ms/step - loss: 5.1793 - val_loss: 9.3679
Epoch 164/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.3423 - val_loss: 9.9879
Epoch 165/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.9190 - val_loss: 8.8794
Epoch 166/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7388 - val_loss: 9.3978
Epoch 167/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.1931 - val_loss: 8.7280
Epoch 168/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8525 - val_loss: 9.4294
Epoch 169/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7700 - val_loss: 8.6943
Epoch 170/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2746 - val_loss: 8.7

4/4 [==============================] - 0s 11ms/step - loss: 4.5937 - val_loss: 10.5616
Epoch 242/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.6426 - val_loss: 10.5830
Epoch 243/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8094 - val_loss: 10.1603
Epoch 244/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7144 - val_loss: 9.9050
Epoch 245/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.4206 - val_loss: 8.9664
Epoch 246/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7720 - val_loss: 10.1973
Epoch 247/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.8568 - val_loss: 10.9221
Epoch 248/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3331 - val_loss: 9.8982
Epoch 249/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.7564 - val_loss: 11.2011
Epoch 250/10000
4/4 [==============================] - 0s 10ms/step - loss: 5.2315 - val_los

4/4 [==============================] - 0s 10ms/step - loss: 4.2225 - val_loss: 9.4692
Epoch 322/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.2177 - val_loss: 9.0734
Epoch 323/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.0872 - val_loss: 9.0598
Epoch 324/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.3780 - val_loss: 9.1080
Epoch 325/10000
4/4 [==============================] - 0s 12ms/step - loss: 4.3788 - val_loss: 9.3448
Epoch 326/10000
4/4 [==============================] - 0s 12ms/step - loss: 4.4059 - val_loss: 9.8012
Epoch 327/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.3725 - val_loss: 9.0945
Epoch 328/10000
4/4 [==============================] - 0s 11ms/step - loss: 4.3455 - val_loss: 10.3571
Epoch 329/10000
4/4 [==============================] - 0s 9ms/step - loss: 4.6283 - val_loss: 10.8143
Epoch 330/10000
4/4 [==============================] - 0s 10ms/step - loss: 4.3314 - val_loss: 10

...

4/4 [==============================] - 0s 12ms/step - loss: 1.7512 - val_loss: 6.8830
Epoch 2312/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.8079 - val_loss: 6.1935
Epoch 2313/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8807 - val_loss: 6.7070
Epoch 2314/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8360 - val_loss: 7.2246
Epoch 2315/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.6148 - val_loss: 6.4083
Epoch 2316/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8799 - val_loss: 6.3097
Epoch 2317/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8783 - val_loss: 7.2048
Epoch 2318/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.5856 - val_loss: 6.1363
Epoch 2319/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.6404 - val_loss: 8.1499
Epoch 2320/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.7245 - val_

4/4 [==============================] - 0s 13ms/step - loss: 1.6573 - val_loss: 5.8395
Epoch 2391/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.7467 - val_loss: 6.8842
Epoch 2392/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.5984 - val_loss: 6.8560
Epoch 2393/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.5278 - val_loss: 8.1185
Epoch 2394/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.6928 - val_loss: 7.3122
Epoch 2395/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.7251 - val_loss: 7.0641
Epoch 2396/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.5909 - val_loss: 7.1250
Epoch 2397/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.5098 - val_loss: 7.0795
Epoch 2398/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.9402 - val_loss: 8.2117
Epoch 2399/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8238 - val_

4/4 [==============================] - 0s 12ms/step - loss: 1.7812 - val_loss: 7.4627
Epoch 2470/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.3903 - val_loss: 7.1690
Epoch 2471/10000
4/4 [==============================] - 0s 14ms/step - loss: 1.4923 - val_loss: 6.8222
Epoch 2472/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.4807 - val_loss: 6.4743
Epoch 2473/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.3438 - val_loss: 6.5562
Epoch 2474/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.3408 - val_loss: 7.2645
Epoch 2475/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.7103 - val_loss: 6.3737
Epoch 2476/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.5454 - val_loss: 6.6475
Epoch 2477/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.8230 - val_loss: 5.9552
Epoch 2478/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.6619 - val_

4/4 [==============================] - 0s 12ms/step - loss: 1.6001 - val_loss: 6.3660
Epoch 2549/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.6181 - val_loss: 7.8934
Epoch 2550/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.6182 - val_loss: 7.3728
Epoch 2551/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.5906 - val_loss: 7.0837
Epoch 2552/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.3655 - val_loss: 6.7427
Epoch 2553/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.3946 - val_loss: 6.4115
Epoch 2554/10000
4/4 [==============================] - 0s 13ms/step - loss: 1.4243 - val_loss: 7.5892
Epoch 2555/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.6946 - val_loss: 8.3405
Epoch 2556/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.3540 - val_loss: 6.7538
Epoch 2557/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.2348 - val_

4/4 [==============================] - 0s 11ms/step - loss: 1.4200 - val_loss: 7.6554
Epoch 2628/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.3821 - val_loss: 6.5706
Epoch 2629/10000
4/4 [==============================] - 0s 12ms/step - loss: 1.1190 - val_loss: 8.0926
Epoch 2630/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.2737 - val_loss: 7.4100
Epoch 2631/10000
4/4 [==============================] - 0s 10ms/step - loss: 1.4252 - val_loss: 6.3922
Epoch 2632/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.3260 - val_loss: 6.9619
Epoch 2633/10000
4/4 [==============================] - 0s 11ms/step - loss: 1.6339 - val_loss: 6.2510
Epoch 2634/10000
4/4 [==============================] - 0s 10ms/step - loss: 1.5857 - val_loss: 6.1601
Epoch 2635/10000
4/4 [==============================] - 0s 10ms/step - loss: 1.5060 - val_loss: 5.7341
Epoch 2636/10000
4/4 [==============================] - 0s 14ms/step - loss: 1.3819 - val_

In [36]:
def mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    results_data = []

    for i in range(len(test_target)):
        prediction = model.predict(test_input[i:i+1])
        target = test_target[start_target + i]
        error = np.abs(prediction - target)
        errors.append(error)
        error_percent.append(error/target)
        results_data.append([f"Month-{i + 1}", 
                             prediction[0][0], 
                             target, 
                             error[0][0]])

    df_results = pd.DataFrame(results_data, columns=["Month", "Prediction", "Target", "Error"])

    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return df_results, mae, mape

In [37]:
df_results, mae, mape = mae_mape_calculator(trained_model, 
                                            reshaped_test, 
                                            reshaped_test_target, 
                                            start_index)
pd.set_option('display.max_columns', None)
df_results.T

1/1 [==============================] - 0s 15ms/step


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29
Month,Month-1,Month-2,Month-3,Month-4,Month-5,Month-6,Month-7,Month-8,Month-9,Month-10,Month-11,Month-12,Month-13,Month-14,Month-15,Month-16,Month-17,Month-18,Month-19,Month-20,Month-21,Month-22,Month-23,Month-24,Month-25,Month-26,Month-27,Month-28,Month-29,Month-30
Prediction,12.11031,12.341373,12.427139,12.322193,12.077255,11.8515,11.517047,10.919878,10.551701,10.490733,11.157448,11.674272,11.774457,11.196046,10.995986,11.511713,11.901855,11.151794,9.809006,9.394667,9.541817,9.984444,10.580414,11.13278,11.536486,11.924506,12.131495,12.010418,11.793741,11.617157
Target,12.399,11.268,8.741,12.358,11.876,11.713,11.449,7.805,7.726,8.516,8.503,10.175,11.775,9.724,10.069,13.93,11.918,11.757,9.691,7.378,6.97,7.612,8.442,8.835,11.251,11.342,13.11,13.195,10.053,12.274
Error,0.288691,1.073374,3.686139,0.035807,0.201255,0.138499,0.068047,3.114878,2.825701,1.974733,2.654448,1.499271,0.000543,1.472046,0.926986,2.418287,0.016146,0.605206,0.118006,2.016667,2.571817,2.372444,2.138413,2.29778,0.285485,0.582506,0.978505,1.184582,1.740741,0.656843


In [38]:
display(mae)
display(mape)

1.3314613

0.14837973

In [39]:
def year_mae_mape_calculator(model, test_input, test_target, start_target):
    
    errors = []
    error_percent = []
    
    target_sum = 0
    prediction_sum = 0
    
    for i in range(len(test_target)):
        if i % 12 == 0 and i != 0:
            error = np.abs(target_sum - prediction_sum)
            errors.append(error)
            error_percent.append(error / target_sum)
            print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
            target_sum = 0
            prediction_sum = 0
            
        prediction = model.predict(test_input[i:i+1])
        target_sum += test_target[start_target + i]
        prediction_sum += prediction
        
    error = np.abs(target_sum - prediction_sum)
    errors.append(error)
    error_percent.append(error / target_sum)
    print(f"Ano-{i%12}: |Prediction{prediction_sum} - Target[{target_sum}]| =  Error: {error}; MAPE:{abs(prediction_sum - target_sum)/target_sum}")
        
    mae = np.mean(errors)
    mape = np.mean(error_percent) 

    return errors, mae, mape

In [40]:
errors, mae, mape = year_mae_mape_calculator(trained_model, 
                                             reshaped_test, 
                                             reshaped_test_target, 
                                             start_index)
display(errors)
display(mae)
display(mape)

1/1 [==============================] - 0s 15ms/step
Ano-0: |Prediction[[139.44084]] - Target[122.52900000000001]| =  Error: [[16.911842]]; MAPE:[[0.13802318]]
1/1 [==============================] - 0s 16ms/step
Ano-0: |Prediction[[128.97498]] - Target[118.101]| =  Error: [[10.873978]]; MAPE:[[0.09207354]]
1/1 [==============================] - 0s 19ms/step
Ano-5: |Prediction[[71.0138]] - Target[71.22500000000001]| =  Error: [[0.2111969]]; MAPE:[[0.00296521]]


[array([[16.911842]], dtype=float32),
 array([[10.873978]], dtype=float32),
 array([[0.2111969]], dtype=float32)]

9.332339

0.077687316